In [3]:
### Generate a tileConfig for BigStitcher

import json
from itertools import product, repeat, chain

# from https://docs.python.org/3/library/itertools.html
def ncycles(iterable, n):
    "Returns the sequence elements n times"
    return chain.from_iterable(repeat(tuple(iterable), n))

params_file = "/scratch/hoerl/small_spheroid_3/params1.json"
tile_conf_file = "/scratch/hoerl/small_spheroid_3/tileConf_withErr.txt"

with open(params_file, 'r') as fd:
    params = json.load(fd)

with open(tile_conf_file, 'w') as fd:
    fd.writelines('dim=3\n')
    for (i,(x,y)) in enumerate(ncycles(product(range(len(params['x_locs'])), range(len(params['y_locs']))),6)):
        x_loc=params['x_locs'][x] // params['downsampling']
        y_loc=params['y_locs'][y] // params['downsampling']
        fd.writelines('{};;({},{},0)\n'.format(i,x_loc,y_loc))

In [6]:
### Generate a tileConfig for BigStitcher
## V2: real locs

import sys
import json
from itertools import product, repeat, chain

import numpy as np

# from https://docs.python.org/3/library/itertools.html
def ncycles(iterable, n):
    "Returns the sequence elements n times"
    return chain.from_iterable(repeat(tuple(iterable), n))

params_file = "/scratch/hoerl/small_spheroid_3/params1.json"
tile_conf_file = "/scratch/hoerl/small_spheroid_3/tileConf_Correct.txt"

with open(params_file, 'r') as fd:
    params = json.load(fd)

with open(tile_conf_file, 'w') as fd:
    fd.writelines('dim=3\n')
    for (i,(x,y,z)) in enumerate(ncycles(product(range(len(params['x_locs'])),
                                               range(len(params['y_locs'])),
                                               range(len(params['z_locs']))), 6)):

            x_loc=params['x_locs'][x]
            y_loc=params['y_locs'][y]
            z_loc=params['z_locs'][z]

            loc = [z_loc, y_loc, x_loc]
            field_info = params['fields'][','.join(map(str, (x, y, z)))]
            off = field_info['off']
            min_ = np.array(loc) + np.array(off) - np.ceil(np.array(params['fov_size']) / 2)
            
            min_ = min_ // params['downsampling']

            z_loc = float(min_[0])
            y_loc = float(min_[1])
            x_loc = float(min_[2])

            fd.writelines('{};;({},{},{})\n'.format(i,x_loc,y_loc,z_loc))